In [1]:
import pandas as pd
import numpy as np
# from sklearn.neural_network import MLPClassifier
# from sklearn.metrics import f1_score
from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.metrics.pairwise import euclidean_distances
import string
from sklearn.metrics.pairwise import cosine_similarity
!pip install contractions
import contractions
# from sklearn.preprocessing import StandardScaler
# from sklearn.pipeline import make_pipeline
import nltk
# from nltk.translate import meteor, meteor_score
nltk.download('wordnet')
# import Levenshtein
# from nltk.translate.bleu_score import sentence_bleu
# from nltk.tokenize import word_tokenize

Keyring is skipped due to an exception: 'keyring.backends'
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ryanr\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
df_a = pd.read_csv("data/articles.csv")

In [3]:
df_a[df_a['article_id'] == 108775015]['product_group_name']

0    Garment Upper body
Name: product_group_name, dtype: object

In [4]:
#Define new features
def new_features(col1, col2):

    #Find Euclidean Distance
    edist = []

    #Find cosine similarity
    csim = []

    #Find lengths of sentences (words)
    length1 = []
    length2 = []

    #Find slices of sentences (number of similar words between sentences)
    slices = []

    #Find meteor scores
    mscores = []

    #Find Levenshtein distance
    ldists = []

    #Find BLEU scores
    bleu1 = []
    bleu2 = []
    bleu3 = []

    #Find Levenshtein ratio
    lratios = []

    vectorizer = TfidfVectorizer()
    for (sent1, sent2) in zip(col1, col2):

        #Euclidean Distances
        corpus = [sent1, sent2]
        features = vectorizer.fit_transform(corpus).todense() 
        edist.append(euclidean_distances(features[0], features[1])[0][0])

        #Cosine Similarities
        vectors = vectorizer.fit_transform(corpus).toarray()
        csim.append(cosine_similarity(vectors)[0][1])

        #Lengths of sentences (words)
        length1.append(len(sent1.split()))
        length2.append(len(sent2.split()))

        #Slices
        slice = 0  
        for word1 in sent1.split():
            for word2 in sent2.split():
                if word1 == word2:
                    slice += 1
        slices.append(slice)
  
        #Meteor Scores
        mscore = meteor_score.meteor_score([word_tokenize(sent1)], word_tokenize(sent2))
        mscores.append(mscore)

        #Levenshtein Distances
        ldist = Levenshtein.distance(sent1, sent2)
        ldists.append(ldist)

        #Levenshtein Ratios
        lratio = Levenshtein.ratio(sent1, sent2)
        lratios.append(lratio)

        #BLEU Scores
        bscore1 = sentence_bleu([sent1.split()], sent2.split(), weights=[1])
        bleu1.append(bscore1)

        bscore2 = sentence_bleu([sent1.split()], sent2.split(), weights=[1/2, 1/2])
        bleu2.append(bscore2)

        bscore3 = sentence_bleu([sent1.split()], sent2.split(), weights=[1/3, 1/3, 1/3])
        bleu3.append(bscore3)

    #Find absolute value of difference between lengths
    lengthdiff = abs(np.array(length2) - np.array(length1))

    return edist, csim, length1, length2, lengthdiff, slices, mscores, ldists, lratios, bleu1, bleu2, bleu3

#Create df with new features given two string arrays
def new_df(col1, col2):
    edist, csim, length1, length2, lengthdiff, slices, mscores, ldists, lratios, bleu1, bleu2, bleu3 = new_features(col1, col2) 

    return_df = pd.DataFrame()
    return_df['edist'] = edist
    return_df['csim'] = csim
    return_df['length1'] = length1
    return_df['length2'] = length2
    return_df['lengthdiff'] = lengthdiff
    return_df['slices'] = slices
    return_df['mscores'] = mscores
    return_df['ldists'] = ldists
    return_df['lratios'] = lratios
    return_df['bleu1'] = bleu1
    return_df['bleu2'] = bleu2
    return_df['bleu3'] = bleu3

    return return_df

# import warnings

# warnings.filterwarnings(action='ignore')

# new_df()

In [5]:
#Clean the strings to remove punctation, remove common words, and lowercase
def clean_string(text):
    text = text.replace('’', '\'')
    text = contractions.fix(text)
    text = ''.join([word for word in text if word not in string.punctuation])
    text = text.lower()
    return text

# gar_up_body = df_a[df_a['product_group_name'] == 'Garment Upper body'].dropna()

# clean_desc = df_a[df_a['product_group_name'] == 'Garment Upper body'].copy().dropna()[['article_id']]
# clean_desc['clean_desc'] = list(map(clean_string, gar_up_body['detail_desc']))
# clean_desc

In [6]:
def get_cos_sim(product_group_name):
    pgn_articles = df_a[df_a['product_group_name'] == product_group_name].dropna()
    clean_desc = pgn_articles[['article_id']]
    clean_desc['clean_desc'] = list(map(clean_string, pgn_articles['detail_desc']))
    
    tf = TfidfVectorizer(analyzer='word', ngram_range=(1, 3), min_df=0, stop_words='english')
    tfidf_matrix = tf.fit_transform(clean_desc['clean_desc'])
    cos_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
    
    return cos_sim

In [7]:
df_a['product_group_name'].value_counts()

Garment Upper body       42741
Garment Lower body       19812
Garment Full body        13292
Accessories              11158
Underwear                 5490
Shoes                     5283
Swimwear                  3127
Socks & Tights            2442
Nightwear                 1899
Unknown                    121
Underwear/nightwear         54
Cosmetic                    49
Bags                        25
Items                       17
Furniture                   13
Garment and Shoe care        9
Stationery                   5
Interior textile             3
Fun                          2
Name: product_group_name, dtype: int64

In [8]:
# temp = get_cos_sim('Fun') # change product_group_name, save temp
# np.savetxt('fun_cos_sim.txt', temp, fmt='%1.6f') # save cos_sim matrix as .txt file

In [9]:
# clean_desc.set_index('article_id', inplace=True)
# name = pd.Series(clean_desc.index)
# name

In [10]:
# input article_id and appropriate cos_sim matrix, get top 10 recommendations
def recommendations(article_id, cos_sim):
    pgn = df_a[df_a['article_id'] == article_id].reset_index()['product_group_name'][0]

    pgn_articles = df_a[df_a['product_group_name'] == pgn].dropna()
    clean_desc = pgn_articles[['article_id']]
    clean_desc['clean_desc'] = list(map(clean_string, pgn_articles['detail_desc']))
    
    clean_desc.set_index('article_id', inplace=True)
    name = pd.Series(clean_desc.index)
    
    recommended_article = []
    idx = name[name == article_id].index[0]
    score_series = pd.Series(cos_sim[idx]).sort_values(ascending = False)
    top_10_indexes = list(score_series.iloc[1:11].index)
    for i in top_10_indexes:
        recommended_article.append(list(clean_desc.index)[i])
    return recommended_article

In [11]:
temp = np.loadtxt('data/gfb_cos_sim.txt', dtype=float)
temp

array([[1.      , 0.00191 , 0.014937, ..., 0.002238, 0.01441 , 0.010219],
       [0.00191 , 1.      , 0.026466, ..., 0.004008, 0.005439, 0.005437],
       [0.014937, 0.026466, 1.      , ..., 0.004262, 0.040018, 0.016975],
       ...,
       [0.002238, 0.004008, 0.004262, ..., 1.      , 0.003592, 0.013798],
       [0.01441 , 0.005439, 0.040018, ..., 0.003592, 1.      , 0.016687],
       [0.010219, 0.005437, 0.016975, ..., 0.013798, 0.016687, 1.      ]])

In [12]:
temp_pgn = df_a[df_a['product_group_name'] == 'Garment Full body'].dropna()['article_id']

recs_df = pd.DataFrame(columns=['article_id', 'recommendations'])
for aid in temp_pgn:
    recs_df.loc[len(recs_df.index)] = [aid, recommendations(aid, temp)]

c:\Users\ryanr\anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [13]:
recs_df.to_csv('gfb_recs.csv')

In [ ]:
# vectorizer = TfidfVectorizer()

# import itertools
# lst1 = vectorizer.fit_transform(clean_desc).toarray()
# lst2 = vectorizer.fit_transform(clean_desc).toarray()
# lst3 = list(map(lambda x: np.dot(x[0], x[1])/(np.linalg.norm(x[0])*np.linalg.norm(x[1])) , itertools.product(lst1,lst2)))
# lst3

In [ ]:
df_a[df_a['product_group_name'] == 'Garment Upper body']

In [ ]:
df_a['product_group_name']